In [1]:
import os
import pandas as pd
import scanpy as sc
import pyarrow.dataset as ds
import gcsfs
import numpy as np
import dask.array as da
import h5py
import anndata as ad
from tqdm import tqdm
import re

In [ ]:
import sys
print(sys.path)

In [ ]:
# initialize GCS file system for reading data from GCS
fs = gcsfs.GCSFileSystem()
gcs_base_path = "gs://arc-ctc-tahoe100/2025-02-25/" # GCS bucket path

In [ ]:
infile = "/".join([gcs_base_path.rstrip("/"), 'metadata', 'sample_metadata.parquet'])
sample_metadata = ds.dataset(infile, filesystem=fs, format="parquet").head(1500).to_pandas()
sample_metadata[sample_metadata['drug'] == 'Afatinib']

In [ ]:
# helper function to list files 
def get_file_table(gcs_base_path: str, target: str=None, endswith: str=None):
    files = fs.glob("/".join([gcs_base_path.rstrip("/"), "**"]))
    if target:
        files = [f for f in files if os.path.basename(f) == target]
    elif endswith:
        files = [f for f in files if f.endswith(endswith)]
    file_list = []
    for f in files:
        file_list.append(f.split("/")[-2:-1] + [f])
    return pd.DataFrame(file_list, columns=["parent", "final_address"])

In [ ]:
List_of_files = get_file_table(gcs_base_path)

In [ ]:
import subprocess

for i in range(3, 15):
    filename = f"plate{i}_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad"
    source = f"gs://arc-ctc-tahoe100/2025-02-25/h5ad/{filename}"
    destination = f"Data/{filename}"
    subprocess.run(["gsutil", "cp", source, destination], check=True)

In [ ]:
import time
import scipy.sparse as sp

for i in tqdm(range(4, 15)):
    plate =f"plate{i}"
    filename = f"Data/{plate}_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad"
    print(f"Starting: {filename}")
    file_start_time = time.time()
    adata = sc.read_h5ad(filename, backed = "r")
    adata.obs['pseduobulk_group'] = adata.obs['cell_name'].astype(str) + "_" + adata.obs['drugname_drugconc'].astype(str)
    unique_groups = adata.obs['pseduobulk_group'].unique()
    # Initialize a zero-matrix to accumulate pseudobulk counts
    pseudobulk_matrix = np.zeros((len(unique_groups), adata.shape[1]), dtype=np.float32)
    BATCH_SIZE = 600000
    start_time = time.time()
    # Process in chunks
    for i in range(0, adata.n_obs, BATCH_SIZE):
        batch_start_time = time.time()
        batch = adata[i : i + BATCH_SIZE, :]  # Load chunk from disk
        batch_X = batch.X  # Keep sparse
        
        # Get sample IDs for this batch
        batch_groups = batch.obs['pseduobulk_group']
    
        # Aggregate gene counts per sample
        for j, group in enumerate(unique_groups):
            mask = batch_groups == group
            if np.any(mask):
                group_data = batch_X[mask, :]  # Sparse matrix of the group
                pseudobulk_matrix[j, :] += group_data.sum(axis=0).A1  # Convert to dense array for summing
    
        # End timer for the current batch
        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time
        print(f"Processed batch {i // BATCH_SIZE + 1}/{(adata.n_obs // BATCH_SIZE)+1} in {batch_time:.2f} seconds")
    
    # Save the aggregated pseudobulk matrix into the backed H5AD file
    pseudobulk_adata = ad.AnnData(
        X=pseudobulk_matrix,  # Gene expression matrix
        obs=pd.DataFrame({'pseudobulk_group': unique_groups}),  # Sample metadata (index = sample names)
        var=adata.var,  # Gene metadata (index = gene names)
    )
    
    # Save in backed mode
    pseudobulk_adata.write(f'Data/{plate}.h5ad', compression='gzip')
    file_end_time = time.time()
    file_time = file_end_time - file_start_time
    print(f"Saved: {filename} in {file_time:.2f} seconds")
print("Pseudobulk analysis saved successfully in backed mode!")

In [ ]:

adata_list = []
file_list = [f"plate{i}.h5ad" for i in range(1, 15)]
for file in file_list:
    file_path = os.path.join("Data", file)
    print(f"Loading {file_path} ...")
    
    adata = sc.read_h5ad(file_path, backed = 'r')  # Read in backed mode to avoid excessive RAM usage
    adata_list.append(adata)

# Concatenate all AnnData objects while appending cells
adata_combined = adata_list[0].concatenate(*adata_list[1:], join="inner")
print(f"Shape after concatenation: {adata_combined.shape}")

# Adding metadata to the anndata 

adata_combined.obs.index = adata_combined.obs['pseudobulk_group'].astype(str) + adata_combined.obs['batch'].astype(str)
adata_combined.obs['batch'] = 'plate_' + (adata_combined.obs['batch'].astype(int) + 1).astype(str)
adata_combined.obs['cell_name'] =  adata_combined.obs['pseudobulk_group'].str.extract(r"^(.*?)_\[")
adata_combined.obs['drug_name'] =  adata_combined.obs['pseudobulk_group'].str.extract(r"\[\('([^']+)',")
adata_combined.obs['concentration, uM'] =  adata_combined.obs['pseudobulk_group'].str.extract(r"\(\s*'[^']+',\s*([\d\.]+),")

# Add lineage to the obs dataframe: 
cell_info = pd.read_csv("Data/cell_line.csv")
cell_to_lineage = dict(zip(cell_info["Cell_line"], cell_info["lineage"]))
adata_combined.obs["lineage"] = adata_combined.obs["cell_name"].map(cell_to_lineage)

print("Added metadata to the Anndata object")

adata_combined.write(f'Data/all_plates.h5ad', compression='gzip')

print("Saved concatenated file")

In [4]:
adata = sc.read_h5ad('/home/pranayagarwal/Documents/tahoe_100M/Data/all_plates.h5ad', backed = "r")

In [7]:
adata.obs.to_csv("/home/pranayagarwal/Documents/tahoe_100M/Data/meta_data.csv")